Taller 1 Curso: Fundamentos de Programación

* Docente: Daniel Escobar
* Estudiante: Danilo Rodriguez Arango 
* Tema: Objetos y Clases (Simulaciones Usando Python)

Ejercicio 1. **Juego de dados secretos:**
    - Existen N jugadores, cada uno con 5 dados de 6 lados.
    - Cada jugador revuelve sus dados y los tira, cada uno puede ver su propio resultado pero no el de los demás.
    - Es el turno de un jugador, este debe debe decir cuántos dados con qué número hay en la mesa y retar a otro jugador.
    - El jugador retado debe decir si lo que dice el jugador anterior es falso o si puede ser verdadero e incluso un puede haber un número superior de dados del mismo valor.
    - Su código debe simular este escenario y calcular la probabilidad de que el jugador retante esté diciendo algo falso. Debe realizar una función que reciba una estimación y calcule la probabilidad de falso.

In [ ]:
import random
from dataclasses import dataclass
from typing import List, Optional

In [ ]:
# Clases 
class Dado:
    def __init__(self, caras: int = 6):
        self.caras = caras

    def lanzar(self) -> int:
        return random.randint(1, self.caras)


@dataclass(frozen=True)
class Mano:
    dados: List[int]

    @classmethod
    def lanzar(cls, n_dados: int = 5, dado: Optional[Dado] = None) -> "Mano":
        dado = dado or Dado()
        return cls([dado.lanzar() for _ in range(n_dados)])

    def contar(self, cara: int) -> int:
        return sum(1 for v in self.dados if v == cara)


@dataclass(frozen=True)
class Afirmacion:
    cantidad: int 
    cara: int    


class Jugador:
    def __init__(self, n_dados: int = 5, caras: int = 6):
        self.n_dados = n_dados
        self._dado = Dado(caras)
        self.mano = Mano.lanzar(n_dados, self._dado)

    def ver_mano(self) -> Mano:
        return self.mano

In [ ]:
class Juego:
    def __init__(
        self,
        n_jugadores: int,
        n_dados_por_jugador: int = 5,
        caras: int = 6,
        estado_aleatorio: Optional[int] = None
    ):
        if estado_aleatorio is not None:
            random.seed(estado_aleatorio)
        self.n_jugadores = n_jugadores
        self.n_dados_por_jugador = n_dados_por_jugador
        self.caras = caras
        self._dado = Dado(caras)
        self.jugadores = [Jugador(n_dados_por_jugador, caras) for _ in range(n_jugadores)]

    def probabilidad_afirmacion_falsa(
        self,
        afirmacion: Afirmacion,
        indice_retante: int,
        iteraciones: int = 100_000,
        estado_aleatorio: Optional[int] = None
    ) -> float:
        if estado_aleatorio is not None:
            random.seed(estado_aleatorio)

        mi_mano = self.jugadores[indice_retante].ver_mano()
        conocidos = mi_mano.contar(afirmacion.cara)

        dados_desconocidos = (self.n_jugadores - 1) * self.n_dados_por_jugador

        falsos = 0
        for _ in range(iteraciones):
            coincidencias_desconocidas = sum(
                1 for _ in range(dados_desconocidos)
                if random.randint(1, self.caras) == afirmacion.cara
            )
            total = conocidos + coincidencias_desconocidas
            if total < afirmacion.cantidad:
                falsos += 1

        return falsos / iteraciones

In [ ]:
# Simulación asignando valores
if __name__ == "__main__":
    n_jugadores = 5
    n_dados_por_jugador = 5

    juego = Juego(
        n_jugadores=n_jugadores,
        n_dados_por_jugador=n_dados_por_jugador,
        estado_aleatorio=42
    )

    # Jugador retante
    retante = 0
    mi_mano = juego.jugadores[retante].ver_mano().dados
    print(f"Mano del retante: {mi_mano}")

    # Afirmación
    afirmacion = Afirmacion(cantidad=7, cara=3)

    # Probabilidad de que el retante miente
    p_falsa = juego.probabilidad_afirmacion_falsa(
        afirmacion,
        indice_retante=retante,
        iteraciones=50_000,
        estado_aleatorio=123
    )
    print(f"Probabilidad de que el retante mienta: {p_falsa:.4f}")


Ejercicio 2. Recorrer un tablero de parqués:
    - Es un juego de parqués con solo N jugadores, cada jugador tiene M fichas.
    - La salida de la cárcel es automática.
    - ¿Cuántos turnos se demora en terminar el juego?

In [21]:
import random

# Clases 
class Jugador:
    def __init__(self, nombre, num_fichas):
        self.nombre = nombre
        self.fichas = [0] * num_fichas
        self.fichas_en_meta = 0

    def lanzar_dado(self):
        return random.randint(1, 6)

    def mover_ficha(self):
        for i in range(len(self.fichas)):
            if self.fichas[i] < 68:
                self.fichas[i] += self.lanzar_dado()
                if self.fichas[i] >= 68:
                    self.fichas_en_meta += 1
                break

In [22]:
class Juego:
    def __init__(self, num_jugadores, num_fichas):
        self.jugadores = [Jugador(f"Jugador {i+1}", num_fichas) for i in range(num_jugadores)]
        self.turnos = 0

    def jugar_turno(self):
        for jugador in self.jugadores:
            jugador.mover_ficha()

        self.turnos += 1

    def juego_terminado(self):
        return all(jugador.fichas_en_meta == len(jugador.fichas) for jugador in self.jugadores)

    def simular_juego(self):
        while not self.juego_terminado():
            self.jugar_turno()
        return self.turnos

In [23]:
# Parámetros del juego
num_jugadores = 4
num_fichas = 4

# Simulación
juego = Juego(num_jugadores, num_fichas)
turnos = juego.simular_juego()

print(f"El juego terminó en {turnos} turnos.")


El juego terminó en 79 turnos.


Ejercio 3. **Gana el jugador de la carta más grande:**
    - Hay 2 jugadores.
    - Cada uno tiene un [juego stándar de 52 cartas]
    - En cada turno, ambos jugadores muestran una carta.
    - El ganador del turno es quien tenga la carta más grande y se lleva la carta del otro jugador.
    - Si hay empates, se sacan cartas de nuevo hasta desempatar, el ganador del desempate se lleva todas las cartas lanzadas hasta el momento.
    - El juego se acaba cuando un jugador se queda con las 104 cartas.
    - ¿Cuántos turnos tarda en terminar el juego?

In [ ]:
import random

# Clases
class Carta:
    def __init__(self, valor):
        self.valor = valor

    def __repr__(self):
        return f"{self.valor}"

class Jugador:
    def __init__(self, nombre):
        self.nombre = nombre
        self.cartas = [] 
        self.cartas_ganadas = []

    def tomar_cartas(self, cartas):
        self.cartas = cartas

    def jugar_carta(self):
        return self.cartas.pop(0)

    def recibir_cartas(self, cartas):
        self.cartas_ganadas.extend(cartas)

In [ ]:
# Parametros
def crear_mazo():
    valores = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
    mazo = [Carta(valor) for valor in valores for _ in range(4)] # no tengo en cuenta las pintas solo las 4 cartas por número/letra
    return mazo

def valor_carta(carta):
    valores = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7, "8": 8, "9": 9, "10": 10, "J": 11, "Q": 12, "K": 13, "A": 14}
    return valores[carta.valor]

In [ ]:
# Función para simular el juego
def jugar_partida():
    mazo = crear_mazo() + crear_mazo()
    random.shuffle(mazo)

    jugador1 = Jugador("Jugador 1")
    jugador2 = Jugador("Jugador 2")

    jugador1.tomar_cartas(mazo[:52])
    jugador2.tomar_cartas(mazo[52:])

    turnos = 0 

    while len(jugador1.cartas) > 0 and len(jugador2.cartas) > 0:
        turnos += 1
        cartas_en_juego = [] 

        carta1 = jugador1.jugar_carta()
        carta2 = jugador2.jugar_carta()
        cartas_en_juego.append(carta1)
        cartas_en_juego.append(carta2)

        # Comparar las cartas
        while valor_carta(carta1) == valor_carta(carta2):
            if len(jugador1.cartas) == 0 or len(jugador2.cartas) == 0:
                break
            carta1 = jugador1.jugar_carta()
            carta2 = jugador2.jugar_carta()
            cartas_en_juego.append(carta1)
            cartas_en_juego.append(carta2)

        if valor_carta(carta1) > valor_carta(carta2):
            jugador1.recibir_cartas(cartas_en_juego)
        elif valor_carta(carta1) < valor_carta(carta2):
            jugador2.recibir_cartas(cartas_en_juego)

    # Determinar el ganador
    if len(jugador1.cartas) == 0:
        return jugador2.nombre, turnos
    else:
        return jugador1.nombre, turnos

ganador, turnos = jugar_partida()
print(f"El ganador es {ganador} y el juego tardó {turnos} turnos.")